In [1]:
!pip install python-dotenv

In [48]:
%set_env CLIENT_ID= "d11f46cce25b48f49d3d0ca5995d6c7e"
%set_env CLIENT_SECRET= "6683ebe398394dc4ad758f6b070aeea2"

env: CLIENT_ID="d11f46cce25b48f49d3d0ca5995d6c7e"
env: CLIENT_SECRET="6683ebe398394dc4ad758f6b070aeea2"


In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv

In [6]:
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [39]:
import os
import base64
import json


In [19]:
from requests import post

In [52]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

In [53]:
print(client_id,client_secret)

"d11f46cce25b48f49d3d0ca5995d6c7e" "6683ebe398394dc4ad758f6b070aeea2"


In [54]:
def get_token():
    auth_string = client_id +":" +client_secret
    auth_bytes= auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")
    url = "https://accounts.spotify.com/api/token"
    headers = {
    "Authorization" : "basic" +auth_base64,
    "Content-Type" :"application/x-www-form-urlencoded"
            }

    data ={"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result= json.loads(result.content)
    token =json_result["access_token"]
    return token
    

In [55]:
auth_string = client_id +":" +client_secret
auth_bytes= auth_string.encode("utf-8")
auth_base64 = str(base64.b64encode(auth_bytes),"utf-8")
url = "https://accounts.spotify.com/api/token"
headers = {
"Authorization" : "basic" +auth_base64,
"Content-Type" :"application/x-www-form-urlencoded"}

data ={"grant_type": "client_credentials"}
result = post(url, headers=headers, data=data)
json_result= json.loads(result.content)

In [56]:
json_result

{'error': 'invalid_client'}

In [57]:
auth_url = 'https://accounts.spotify.com/api/token'

In [68]:
client_id = "d11f46cce25b48f49d3d0ca5995d6c7e"
client_secret = "6683ebe398394dc4ad758f6b070aeea2"

In [69]:
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}

In [70]:
import requests
auth_response = requests.post(auth_url, data=data)

In [71]:
access_token = auth_response.json().get('access_token')

In [72]:
print(access_token)

BQBu8GaMlvyun-3ksLoCKZb6l5jsHP5rbOocftDumFB99VAGA7jcuJlFgK4pFDEbEyaNV-x0J8gyiUC4OtlAako0_yhdhYJJzGh-GYlORl07edTjlpw


In [89]:
base_url = 'https://api.spotify.com/v1/'
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}
featured_playlists_endpoint = 'artists/1McMsnEElThX1knmY4oliG'
featured_playlists_url = ''.join([base_url,featured_playlists_endpoint])

In [90]:
response = requests.get(featured_playlists_url,headers=headers)

In [91]:
response

<Response [200]>

In [92]:
response.json()

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1McMsnEElThX1knmY4oliG'},
 'followers': {'href': None, 'total': 28030698},
 'genres': ['pop'],
 'href': 'https://api.spotify.com/v1/artists/1McMsnEElThX1knmY4oliG',
 'id': '1McMsnEElThX1knmY4oliG',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebe03a98785f3658f0b6461ec4',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174e03a98785f3658f0b6461ec4',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178e03a98785f3658f0b6461ec4',
   'width': 160}],
 'name': 'Olivia Rodrigo',
 'popularity': 90,
 'type': 'artist',
 'uri': 'spotify:artist:1McMsnEElThX1knmY4oliG'}

In [93]:
import os
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd

# Define the base URL for Spotify charts
base_url = "https://charts.spotify.com/charts/view/regional-us-weekly/"

# Define the start and end dates
start_date = date(2016, 12, 29)
end_date = date(2023, 9, 14)

# Define the folder where you want to save the CSV files
output_folder = "spotify_charts_csv"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to download CSV for a specific date
def download_csv_for_date(date_str):
    url = base_url + date_str
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        csv_link = soup.find("a", {"class": "chart-csv"}).get("href")
        csv_url = f"https://charts.spotify.com{csv_link}"
        response = requests.get(csv_url)
        if response.status_code == 200:
            with open(os.path.join(output_folder, f"{date_str}.csv"), "wb") as f:
                f.write(response.content)
                print(f"Downloaded {date_str}.csv")
        else:
            print(f"Failed to download {date_str}.csv")
    else:
        print(f"Failed to fetch {url}")

# Loop through the date range and download CSV files
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    download_csv_for_date(date_str)
    current_date += timedelta(days=7)

# Optional: Merge all downloaded CSV files into a single DataFrame
csv_files = [os.path.join(output_folder, f"{date_str}.csv") for date_str in pd.date_range(start=start_date, end=end_date, freq='7D').strftime("%Y-%m-%d")]
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_df.to_csv(os.path.join(output_folder, "combined_spotify_charts.csv"), index=False)

print("All CSV files downloaded and merged.")


AttributeError: 'NoneType' object has no attribute 'get'

In [97]:
import os
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd

# Define the base URL for Spotify charts
base_url = "https://charts.spotify.com/charts/view/regional-us-weekly/"

# Define the start and end dates
start_date = date(2016, 12, 29)
end_date = date(2023, 9, 14)

date_str = current_date.strftime("%Y-%m-%d")

In [98]:
base_url = "https://charts.spotify.com/charts/view/regional-us-weekly/"

In [100]:
url = base_url + date_str
response = requests.get(url)

In [111]:
url

'https://charts.spotify.com/charts/view/regional-us-weekly/2019-11-07'

In [101]:
response

<Response [200]>

In [102]:
soup = BeautifulSoup(response.text, 'html.parser')

In [110]:
soup

<!DOCTYPE html>
<html class="no-touchevents" lang="en"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="/spotifycharts.svg" rel="shortcut icon"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Book.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Bold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Black.woff2" rel="preload" type="font/woff2"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><title>Spotify Charts - Spotify Charts are made by fans</title><meta content="The new home for Spotify Charts. Dive into artist, genre, city and local pulse charts to see what music is moving fans around the world." name="description"/><me

In [129]:
soup.find_all()[10]

<title>Spotify Charts - Spotify Charts are made by fans</title>

In [106]:
soup

<!DOCTYPE html>
<html class="no-touchevents" lang="en"><head><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="/spotifycharts.svg" rel="shortcut icon"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Book.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Bold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="crossorigin" href="https://encore.scdn.co/1.2.3/CircularSpotify-UI-Latin-OS2v3-Black.woff2" rel="preload" type="font/woff2"/><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><title>Spotify Charts - Spotify Charts are made by fans</title><meta content="The new home for Spotify Charts. Dive into artist, genre, city and local pulse charts to see what music is moving fans around the world." name="description"/><me

In [96]:
import os
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd

# Define the base URL for Spotify charts
base_url = "https://charts.spotify.com/charts/view/regional-us-weekly/"

# Define the start and end dates
start_date = date(2016, 12, 29)
end_date = date(2023, 9, 14)

# Define the folder where you want to save the CSV files
output_folder = "spotify_charts_csv"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to download CSV for a specific date
def download_csv_for_date(date_str):
    url = base_url + date_str
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        csv_link = soup.find("a", {"class": "styled__CSVLink-sc-135veyd-5 kMpXks"})
        if csv_link:
            csv_url = f"https://charts.spotify.com{csv_link.get('href')}"
            response = requests.get(csv_url)
            if response.status_code == 200:
                with open(os.path.join(output_folder, f"{date_str}.csv"), "wb") as f:
                    f.write(response.content)
                    print(f"Downloaded {date_str}.csv")
            else:
                print(f"Failed to download {date_str}.csv")
        else:
            print(f"No CSV link found for {date_str}")
    else:
        print(f"Failed to fetch {url}")

# Loop through the date range and download CSV files
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    download_csv_for_date(date_str)
    current_date += timedelta(days=7)

# Optional: Merge all downloaded CSV files into a single DataFrame
csv_files = [os.path.join(output_folder, f"{date_str}.csv") for date_str in pd.date_range(start=start_date, end=end_date, freq='7D').strftime("%Y-%m-%d")]
combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_df.to_csv(os.path.join(output_folder, "combined_spotify_charts.csv"), index=False)

print("All CSV files downloaded and merged.")


No CSV link found for 2016-12-29
No CSV link found for 2017-01-05
No CSV link found for 2017-01-12
No CSV link found for 2017-01-19
No CSV link found for 2017-01-26
No CSV link found for 2017-02-02
No CSV link found for 2017-02-09
No CSV link found for 2017-02-16
No CSV link found for 2017-02-23
No CSV link found for 2017-03-02
No CSV link found for 2017-03-09
No CSV link found for 2017-03-16
No CSV link found for 2017-03-23
No CSV link found for 2017-03-30
No CSV link found for 2017-04-06
No CSV link found for 2017-04-13
No CSV link found for 2017-04-20
No CSV link found for 2017-04-27
No CSV link found for 2017-05-04
No CSV link found for 2017-05-11
No CSV link found for 2017-05-18
No CSV link found for 2017-05-25
No CSV link found for 2017-06-01
No CSV link found for 2017-06-08
No CSV link found for 2017-06-15
No CSV link found for 2017-06-22
No CSV link found for 2017-06-29
No CSV link found for 2017-07-06
No CSV link found for 2017-07-13
No CSV link found for 2017-07-20
No CSV lin

KeyboardInterrupt: 

In [95]:
url

'https://accounts.spotify.com/api/token'

In [ ]:
!pip install selenium

In [137]:
import os
import pandas as pd
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Define the base URL for Spotify charts
base_url = "https://charts.spotify.com/charts/view/regional-us-weekly/"

# Define the start and end dates
start_date = date(2016, 12, 29)
end_date = date(2023, 9, 14)

# Define the folder where you want to save the CSV files
output_folder = "spotify_charts_csv"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Function to download CSV for a specific date
def download_csv_for_date(date_str):
    url = base_url + date_str
    driver = webdriver.Chrome()  # You'll need to install and configure the Selenium WebDriver for your browser
    driver.get(url)
    
    try:
        # Use a more general XPath to locate the button
        csv_button = driver.find_element(By.XPATH, '//button[contains(text(),"Download CSV")]')
        csv_button.click()
        print(f"Downloaded {date_str}.csv")
    except Exception as e:
        print(f"Failed to download {date_str}.csv: {str(e)}")
    
    driver.quit()

# Loop through the date range and download CSV files
current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    download_csv_for_date(date_str)
    current_date += timedelta(days=7)

print("All CSV files downloaded.")


Failed to download 2016-12-29.csv: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(text(),"Download CSV")]"}
  (Session info: chrome=116.0.5845.188); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A57352A2+57122]
	(No symbol) [0x00007FF7A56AEA92]
	(No symbol) [0x00007FF7A557E3AB]
	(No symbol) [0x00007FF7A55B7D3E]
	(No symbol) [0x00007FF7A55B7E2C]
	(No symbol) [0x00007FF7A55F0B67]
	(No symbol) [0x00007FF7A55D701F]
	(No symbol) [0x00007FF7A55EEB82]
	(No symbol) [0x00007FF7A55D6DB3]
	(No symbol) [0x00007FF7A55AD2B1]
	(No symbol) [0x00007FF7A55AE494]
	GetHandleVerifier [0x00007FF7A59DEF82+2849794]
	GetHandleVerifier [0x00007FF7A5A31D24+3189156]
	GetHandleVerifier [0x00007FF7A5A2ACAF+3160367]
	GetHandleVerifier [0x00007FF7A57C6D06+653702]
	(No symbol) [0x00007FF7A56BA208]
	(No symbol) [0x00007FF7A56B62

KeyboardInterrupt: 

'2017-06-29'

Failed to download 2017-06-29.csv: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div/main/div[2]/div[3]/div/div/a"}
  (Session info: chrome=116.0.5845.188); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A57352A2+57122]
	(No symbol) [0x00007FF7A56AEA92]
	(No symbol) [0x00007FF7A557E3AB]
	(No symbol) [0x00007FF7A55B7D3E]
	(No symbol) [0x00007FF7A55B7E2C]
	(No symbol) [0x00007FF7A55F0B67]
	(No symbol) [0x00007FF7A55D701F]
	(No symbol) [0x00007FF7A55EEB82]
	(No symbol) [0x00007FF7A55D6DB3]
	(No symbol) [0x00007FF7A55AD2B1]
	(No symbol) [0x00007FF7A55AE494]
	GetHandleVerifier [0x00007FF7A59DEF82+2849794]
	GetHandleVerifier [0x00007FF7A5A31D24+3189156]
	GetHandleVerifier [0x00007FF7A5A2ACAF+3160367]
	GetHandleVerifier [0x00007FF7A57C6D06+653702]
	(No symbol) [0x00007FF7A56BA208]
	(No symbol) [0x0